# Getting started

This script will explore [Elena Cuoco's script](https://www.kaggle.com/elenacuoco/grasp-and-lift-eeg-detection/simple-grasp-with-sklearn) in an effort to understand what is going on.

In [1]:
# contains important preprocessing tools to read in our data
from tools import *

### Simple Logistic Regression

In [2]:
# read in all the data

all_data = []
all_labels = []

for subject in range(1,5):
    for series in range(1,5):
        data, labels = prepare_data_train(subject, series)
        all_data.append(data)
        all_labels.append(labels)        

In [ ]:
import numpy as np
import pandas as pd

In [3]:
# convert the data and labels into pandas data frames.

X = pd.concat(all_data)
Y = pd.concat(all_labels)

# transform the training data in the numpy array
X_train = np.asarray(X.astype(float))
y = np.asarray(Y.astype(float))



In [13]:
recordings, channels = X_train.shape
print 'we have ' , recordings, ' recordings and ', channels, ' channels each to train on.'

we have  3311437  recordings and  32  channels each to train on.


In [40]:
# read in the test data


for subject in range(1,5):
    test_data = [] 
    idx = [] # holds the ids of the participants
    
    for series in range(9,11):
        data = prepare_data_test(subject, series)
        test_data.append(data)
        idx.append( np.array( data['id'] ) )

    X_test = pd.concat(test_data)
    X_test = X_test.drop(['id'], axis=1) # remove ids
    X_test = np.asarray( X_test.astype(float) )

    ids_tot = np.concatenate(idx) # contains all the ids

In [10]:
recordings, channels = X_test.shape
print 'we have ' , recordings, ' recordings and ', channels, ' channels each to test.'

we have  1001299  recordings and  32  channels each to test.


In [41]:
# classifier training

from sklearn.linear_model import LogisticRegression

# we will use a Logistic Regression Model
lr = LogisticRegression()

# removes the mean and scales the variance to 1
X_train = data_preprocess_train(X_train)

cols = ['HandStart','FirstDigitTouch',\
        'BothStartLoadPhase','LiftOff',\
        'Replace','BothReleased']

subsample = 100

predictions = np.empty( ( X_test.shape[0], 6 ) ) # for the number of recordings, we have 6 labels

all_predictions = []


for subject in range(1,5):
    for i in range(6): # go through each of the columns 
        # select the the i'th column for training
        y_train = y[:,i] # this holds the label
        print 'Train Subject',subject,' class ', cols[i]
        lr.fit( X_train[::subsample,:], y_train[::subsample] ) #sample every 100th item.
        
        predictions[:,i] = lr.predict_proba(X_test)[:,1]
        
    all_predictions.append(predictions)


    


Train Subject 1  class  HandStart
Train Subject 1  class  FirstDigitTouch
Train Subject 1  class  BothStartLoadPhase
Train Subject 1  class  LiftOff
Train Subject 1  class  Replace
Train Subject 1  class  BothReleased
Train Subject 2  class  HandStart
Train Subject 2  class  FirstDigitTouch
Train Subject 2  class  BothStartLoadPhase
Train Subject 2  class  LiftOff
Train Subject 2  class  Replace
Train Subject 2  class  BothReleased
Train Subject 3  class  HandStart
Train Subject 3  class  FirstDigitTouch
Train Subject 3  class  BothStartLoadPhase
Train Subject 3  class  LiftOff
Train Subject 3  class  Replace
Train Subject 3  class  BothReleased
Train Subject 4  class  HandStart
Train Subject 4  class  FirstDigitTouch
Train Subject 4  class  BothStartLoadPhase
Train Subject 4  class  LiftOff
Train Subject 4  class  Replace
Train Subject 4  class  BothReleased


In [42]:
file_name = 'submission1.csv'

submission = pd.DataFrame(index= np.concatenate(ids_tot), \
                         columns = cols, \
                         data = np.concatenate(all_predictions) )
submission.to_csv(file_name, index_label='id', float_format='%.3f')

ValueError: zero-dimensional arrays cannot be concatenated

In [39]:
ids_tot

array(['subj1_series9_0', 'subj1_series9_1', 'subj1_series9_2', ...,
       'subj4_series10_123524', 'subj4_series10_123525',
       'subj4_series10_123526'], dtype=object)